# Example of Pretrained Pipelines

In [ ]:
import json, os
import sys

if 'google.colab' in sys.modules:
    from google.colab import files

    if 'spark_ocr.json' not in os.listdir():
      license_keys = files.upload()
      os.rename(list(license_keys.keys())[0], 'spark_ocr.json')

with open('spark_ocr.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

Saving spark-license-v440.json to spark-license-v440.json


In [ ]:
!pip install transformers

# Installing pyspark and spark-nlp
%pip install --upgrade -q pyspark==3.2.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark OCR
#! pip uninstall spark-ocr -Y
%pip install spark-ocr==$OCR_VERSION --extra-index-url=https://pypi.johnsnowlabs.com/$SPARK_OCR_SECRET --upgrade

<b><h1><font color='darkred'>!!! ATTENTION !!! </font><h1><b>

<b>After running previous cell, <font color='darkred'>RESTART the COLAB RUNTIME </font> and go ahead.<b>

In [ ]:
import json, os

with open("spark_ocr.json", 'r') as f:
  license_keys = json.load(f)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

# Defining license key-value pairs as local variables
locals().update(license_keys)

In [ ]:
import pkg_resources

from pyspark.ml import PipelineModel
import pyspark.sql.functions as f

from sparkocr import start
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import *
from sparkocr.metrics import score

In [ ]:
# Start spark
spark = start(secret=SPARK_OCR_SECRET, nlp_version=PUBLIC_VERSION)

Spark version: 3.2.1
Spark NLP version: 4.3.1
Spark OCR version: 4.4.0rc1



## Import OCR transformers

In [ ]:
from pyspark.ml import PipelineModel
from sparkocr.pretrained import *

mixed_pdf_pipeline = PretrainedPipeline('pdf_handwritten_transformer_extraction', 'en', 'clinical/ocr')

pdf_handwritten_transformer_extraction download started this may take some time.
Approx size to download 494.5 MB
[OK!]


## Call the pipeline

In [ ]:
pdf_path = './data/mixed/handwritten/pdfs'
pdf_example_df = spark.read.format("binaryFile").load(pdf_path).cache()
result = mixed_pdf_pipeline.transform(pdf_example_df)
result

path,modificationTime,length,text,positions,height_dimension,width_dimension,content,image,total_pages,pagenum,documentnum,confidence,exception
file:/content/mix...,2023-03-27 10:03:...,675876,ROMINVENT\n\nINDU...,[{[{ROMINVENT\n\n...,792.0,612.0,[25 50 44 46 2D 3...,{file:/content/mi...,1,0,0,93.3254582087199,null
file:/content/mix...,2023-03-27 10:03:...,237661,...,"[{[{v, 0, 84.182,...",838.0999755859375,594.0,[25 50 44 46 2D 3...,null,0,0,0,-1.79769313486231...,null
